In [176]:
import numpy as np
import h5py
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import h5py

In [177]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import Tensor
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Reshape, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, UpSampling2D, GlobalMaxPooling2D, Conv2DTranspose
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model, load_model
from tensorflow.python.keras.utils import layer_utils
from tensorflow.keras.utils import model_to_dot
from keras.utils.vis_utils import plot_model
from keras.callbacks import CSVLogger
import keras.models
from typing import List, Tuple
from time import time
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
!pip install TensorRT

In [178]:
h = h5py.File('new_reduced_data.h5', 'r')

In [179]:
glitches = h['new_glitches'][:]

In [180]:
glitch_labels = h['glitch_labels'][:]

In [181]:
# separating the instances
norm_data = h['new_clean'][:]
norm_labels = h['clean_labels'][:]
anom1_data = h['new_whistle'][:]
anom1_labels = h['whistle_labels'][:]
anom2_data = h['new_tomte'][:]
anom2_labels = h['tomte_labels'][:]
anom3_data = h['new_sclight'][:]
anom3_labels = h['sclight_labels'][:]

In [182]:
def convolution_1d(X: tf.Tensor, filters: List[int], kernel: Tuple[int,int], s: Tuple[int,int], padding:str, name: str) -> tf.Tensor:
    f1 = filters
    
    X = Conv2D(filters=f1, kernel_size=kernel, strides=s, padding=padding, name=name)(X)
    X = Activation('relu')(X)
    
    return X

In [183]:
def deconvolution_1d(X: tf.Tensor, filters: List[int], kernel: Tuple[int,int], s: Tuple[int,int], padding:str, name: str) -> tf.Tensor:
    f1 = filters
    
    X = Conv2DTranspose(filters=f1, kernel_size=kernel, strides=s, padding=padding, name=name)(X)
    X = Activation('relu')(X)
    
    return X

In [193]:
def Net(input_size: Tuple[int,int,int]) -> Model:
    X_input1 = Input(input_size)
    
    X_input2 = X_input1*1
    
    tensor_shape = tf.shape(X_input2)
    
    X_input2 = tf.slice(X_input2, [0, 0, 0, 0], [-1, tensor_shape[1] - 1, -1, -1])
    
    X = tf.concat((X_input1, X_input2), axis=1)
    
    X = convolution_1d(X, filters=120, kernel=(X_input1.shape[1],1), s=(1, 1), padding='valid', name='convolution_1')
    
    X = convolution_1d(X, filters=150, kernel=(5,1), s=(2, 1), padding='same', name='convolution_2')
    
    X = convolution_1d(X, filters=150, kernel=(5,1), s=(5, 1), padding='same', name='convolution_3')
    
    X = deconvolution_1d(X, filters=120, kernel=(5,1), s=(5, 1), padding='same', name='deconvolution_1')
    
    X = deconvolution_1d(X, filters=1, kernel=(5,1), s=(2, 1), padding='same', name='deconvolution_2')
        
    # Create model
    model = Model(inputs=X_input1, outputs=X, name='ConvAutoEnc')

    return model

In [194]:
model = Net(input_size = (50, 56, 1))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss='mse')
csv_logger = CSVLogger('training.log')
t0 = time()
history = model.fit(glitches, glitches, batch_size=168, epochs=500, callbacks=[csv_logger])
print('Training time: ', time() - t0)

In [205]:
# we can now extract the features of the data
feature_model = Model(inputs=model.input, outputs=model.get_layer(name='convolution_3').output)
features = feature_model.predict(glitches)
print('feature shape=', features.shape)

84/84 [==============================] - 2s 25ms/step
feature shape= (2688, 5, 56, 150)


In [206]:
features1 = np.reshape(features, newshape=(features.shape[0], -1))

In [ ]:
# let us visualize
tSNE = TSNE(n_components=2, learning_rate='auto')
X_tsne = tSNE.fit_transform(features1)

In [ ]:
plot = plt.scatter(X_tsne[:,0], X_tsne[:,1], s=10, c=glitch_labels, alpha=0.7)
#plt.scatter(X_tsne[idxs[:],0], X_tsne[idxs[:],1], s=20, c='red')
plt.legend(handles=plot.legend_elements()[0], labels=['Whistle', 'Tomte', 'Scattered light'])
plt.ylabel('t-SNE dimension 2')
plt.xlabel('t-SNE dimension 1')
plt.title('Embedded space of glitches')
plt.show()